In [17]:
from scipy.optimize import minimize
import numpy as np

In [18]:
# error function parameters  modes
eta =0.06279174
'''rho = pm.Uniform("rho", lower=0, upper=10)
beta_L = pm.Uniform("beta_L", lower=0, upper=3)
beta_H = pm.Uniform("beta_H", lower=0, upper=6)'''
rho = 90.84086656
beta_L = 1.01224394
beta_H = 0.73874365
# rho_2 = pm.Uniform("rho_2", lower=0, upper=1)
beta_H2 = 0.37773169
# alpha_1 = pm.Normal("alpha_1", mu=10, sigma=5)
alpha_L = 3.36892667
alpha_1 = 3.33742461
alpha_2 = -1.55839418

sigma_lg_Phi = 0.08570291 

# computational cost
C_L = .998 + 1.869  # C_L1 + C_L2 in CPU hours
C_H = 178.813

q = C_H / C_L

# objective error
err_0 = .05 # goal error

# production run cost (not used in the optimization, only for the comparison with the single-fidelity counterpart finally)
C_L_prod = (21.4 +44.9)*56 # CPU hours
C_H_prod = 4452*56 # CPU hours


In [19]:
def err_func(n_LF, n_HF):
    Phi = eta * (rho / ( (n_LF + n_HF) * (n_LF + alpha_L)**(beta_L-1) * (n_HF + alpha_1)**(beta_H-1) ) + 1 / (alpha_2 + n_HF)**(beta_H2))
    # median_Phi = 10**lg_Phi
    return Phi

def err_mean(n_LF, n_HF):
    gamma = np.exp(1/2 * (sigma_lg_Phi * np.log(10))**2)
    return gamma * err_func(n_LF, n_HF)

def comp_bud(n_L, n_H):
    return C_L * n_L + C_H * n_H

def comp_bud_prod(n_L, n_H):   # for production runs
    return C_L_prod * n_L + C_H_prod * n_H

In [20]:


# # Define the objective function
def objective(vars):
    n_L, n_H = vars
    return comp_bud(n_L, n_H)

# Define the constraint function
def constraint(vars):
    n_L, n_H = vars
    return err_mean(n_L,n_H) - err_0

# Initial guess for the variables
initial_guess = [600, 10]

# Define the constraint dictionary
con = {'type': 'eq', 'fun': constraint}

# Perform the optimization
result = minimize(objective, initial_guess, constraints=[con])

# Extract the optimal values of x and y
x_opt, y_opt = result.x

print(f"Optimal values: x = {x_opt}, y = {y_opt}")
print(f"Function value at optimal points: f(x, y) = {objective((x_opt, y_opt))}")

Optimal values: x = 583.9282026211353, y = 7.278585048292925
Function value at optimal points: f(x, y) = 2975.6277851551977


In [21]:
comp_bud(x_opt, y_opt) / 56

53.13621044919996

In [22]:
comp_bud(x_opt, y_opt) / 56 * 1200

63763.45253903995

In [23]:
n_LF = 564
n_HF = 21

In [24]:
err_mean(n_LF,n_HF)

0.04205262414022338

In [25]:
comp_bud(n_LF, n_HF) / 56 * 1200

115115.59285714284

In [26]:
err_mean(0, 564)

0.059110607942079424

In [27]:
err_mean(0, 564*1.5)

0.04447272109987826

In [28]:
comp_bud(0, 564) / 56 * 1200

2161082.8285714285

In [29]:
err_func(10000, 0)

(0.020598159424718255-0.049233718543544386j)

In [30]:
C_singlef = comp_bud_prod(0, 564) / 56

In [31]:
C_mfbox = comp_bud_prod(564, 21) / 56
# print node hours
print('C_singlef: ', C_singlef, 'node hours')
print('C_mfbox: ', C_mfbox, 'node hours')
print('Single-fidelity is ', C_singlef / C_mfbox, ' times more expensive than MF-box')
print("In another word, MF-Box reduces the computational cost by %f" % (100*(1 - C_mfbox/C_singlef)) + "%")


C_singlef:  2510928.0 node hours
C_mfbox:  130885.2 node hours
Single-fidelity is  19.1842011167038  times more expensive than MF-box
In another word, MF-Box reduces the computational cost by 94.787377%
